<a href="https://colab.research.google.com/github/rangelokk/Diplom/blob/main/Keras_ResNet_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Residual Networks

Welcome to another tutorial! Now we will learn how to build very deep convolutional networks, using Residual Networks (ResNets).

**In this assignment, we will:**
- Implement the basic building blocks of ResNets.
- Put together these building blocks to implement and train a state-of-the-art neural network for image classification.

Let's run the cell below to load the required packages:

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import cv2
import numpy as np
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D
from keras.models import Model, load_model
from keras.initializers import glorot_uniform
from keras.utils import plot_model
from IPython.display import SVG
#from keras.utils.vis_utils import model_to_dot
from tensorflow.keras.utils import plot_model
import keras.backend as K
import tensorflow as tf

In [ ]:
import kagglehub
import numpy as np
import seaborn as sns
import pandas as pd
import imgaug.augmenters as iaa
import imageio
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, f1_score, mean_squared_error, mean_absolute_error
from sklearn.linear_model import SGDClassifier
from PIL import Image

In [ ]:
ROWS = 64
COLS = 64
CHANNELS = 3
CLASSES = 7

### Загрузка данных

In [ ]:
# Download latest version
path = kagglehub.dataset_download("kmader/skin-cancer-mnist-ham10000")

print("Path to dataset files:", path)

100%|██████████| 5.20G/5.20G [00:52<00:00, 107MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/kmader/skin-cancer-mnist-ham10000/versions/2


In [ ]:
data = pd.read_csv(path + "/HAM10000_metadata.csv")
data.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear


In [ ]:
def read_image(file_path):
  img = cv2.imread(path + '/HAM10000_images_part_2/'+file_path+'.jpg', cv2.IMREAD_COLOR)
  if (img is None):
    img = cv2.imread(path + '/HAM10000_images_part_1/'+file_path+'.jpg', cv2.IMREAD_COLOR)
  return cv2.resize(img, (ROWS, COLS), interpolation=cv2.INTER_CUBIC)

def prepare_data(images):
    m = len(images)
    print(m)
    X = np.zeros((m, ROWS, COLS, CHANNELS), dtype=np.uint8)
    y = np.zeros((1, m), dtype=np.uint8)
    for i, image_file in enumerate(images):
        X[i,:] = read_image(image_file)
    return X

def convert_to_one_hot(Y, C):
    Y = np.eye(C)[Y.reshape(-1)].T
    return Y

Run the following code to normalize the dataset and learn about its shapes:

In [ ]:
X = prepare_data(data['image_id'])

10015


In [ ]:
X.shape

(10015, 64, 64, 3)

In [ ]:
X_train, X_test = train_test_split(X/255., test_size=0.25, random_state=42)
class_labels = {
    'akiec': 0,
    'bcc': 1,
    'bkl': 2,
    'df': 3,
    'nv': 4,
    'vasc': 5,
    'mel': 6
}
train_set_y, test_set_y = train_test_split(data['dx'].replace(class_labels), test_size=0.25, random_state=42)

Y_train = convert_to_one_hot(train_set_y.to_numpy(), CLASSES).T
Y_test = convert_to_one_hot(test_set_y.to_numpy(), CLASSES).T

<ipython-input-9-45d551131f3e>:11: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_set_y, test_set_y = train_test_split(data['dx'].replace(class_labels), test_size=0.25, random_state=42)


In [ ]:

print ("number of training examples =", X_train.shape[0])
print ("number of test examples =", X_test.shape[0])
print ("X_train shape:", X_train.shape)
print ("Y_train shape:", Y_train.shape)
print ("X_test shape:", X_test.shape)
print ("Y_test shape:", Y_test.shape)

number of training examples = 7511
number of test examples = 2504
X_train shape: (7511, 64, 64, 3)
Y_train shape: (7511, 7)
X_test shape: (2504, 64, 64, 3)
Y_test shape: (2504, 7)


## 2 - Building a Residual Network

In ResNets, a "shortcut" or a "skip connection" allows the gradient to be directly backpropagated to earlier layers:  

<img src="images/skip_connection.png" style="width:650px;height:200px;">

Two main types of blocks are used in a ResNet, depending mainly on whether the input/output dimensions are same or different. We are going to implement both of them.

#### Why do Skip Connections work?
- They mitigate the problem of vanishing gradient by allowing this alternate shortcut path for gradient to flow through
- They allow the model to learn an identity function which ensures that the higher layer will perform at least as good as the lower layer, and not worse

### 2.1 - The identity block

We'll implement identity block, in which the skip connection "skips over" 3 hidden layers rather than 2 layers. It looks like this:

<img src="images/Identity_block2.png" style="width:650px;height:150px;">


**Arguments**:<br>
    X - input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)<br>
    f - integer, specifying the shape of the middle CONV's window for the main path<br>
    filters - python list of integers, defining the number of filters in the CONV layers of the main path<br>
    stage - integer, used to name the layers, depending on their position in the network<br>
    block - string/character, used to name the layers, depending on their position in the network<br>

**Returns**:<br>
    X - output of the identity block, tensor of shape (n_H, n_W, n_C)<br>

In [ ]:
def identity_block(X, f, filters, stage, block):
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    # Retrieve Filters
    F1, F2, F3 = filters

    # Save the input value. We'll need this later to add back to the main path.
    X_shortcut = X

    # First component of main path
    X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    # Second component of main path
    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path
    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    return X

In [ ]:
# Создание входного тензора
A_prev = tf.keras.Input(shape=(4, 4, 6))  # Указываем размер входа (вместо placeholder)

# Применяем identity_block
A = identity_block(A_prev, f=2, filters=[2, 4, 6], stage=1, block='a')

# Создаем модель
model = tf.keras.Model(inputs=A_prev, outputs=A)

# Генерируем случайный ввод
X = np.random.randn(3, 4, 4, 6).astype(np.float32)

# Получаем выход модели
out = model(X)

# Печатаем результат
print("out = ", out[1][0])  # Измените индексы в зависимости от нужного выхода

out =  tf.Tensor(
[[0.6243977  0.         0.5264121  1.7362173  0.         0.        ]
 [1.2458789  0.29965872 0.         0.43595922 0.         0.        ]
 [0.         0.         1.3851578  0.2482921  0.         0.65151286]
 [0.         0.         0.         0.7655969  0.01244041 0.        ]], shape=(4, 6), dtype=float32)


## 2.2 - The convolutional block

The ResNet "convolutional block" is the other type of block:

<img src="images/Identity_block3.png" style="width:650px;height:150px;">

    
**Arguments**:<br>
    X - input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)<br>
    f - integer, specifying the shape of the middle CONV's window for the main path<br>
    filters - python list of integers, defining the number of filters in the CONV layers of the main path<br>
    stage - integer, used to name the layers, depending on their position in the network<br>
    block - string/character, used to name the layers, depending on their position in the network<br>
    s - Integer, specifying the stride to be used<br>

**Returns**:<br>
    X - output of the convolutional block, tensor of shape (n_H, n_W, n_C)

In [ ]:
def convolutional_block(X, f, filters, stage, block, s = 2):
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    # Retrieve Filters
    F1, F2, F3 = filters

    # Save the input value
    X_shortcut = X


    ##### MAIN PATH #####
    # First component of main path
    X = Conv2D(F1, (1, 1), strides = (s,s), name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    # Second component of main path
    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path
    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2c', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)


    ##### SHORTCUT PATH ####
    X_shortcut = Conv2D(F3, (1, 1), strides = (s,s), name = conv_name_base + '1', kernel_initializer = glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis = 3, name = bn_name_base + '1')(X_shortcut)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    return X

In [ ]:
# Создание входного тензора
A_prev = tf.keras.Input(shape=(4, 4, 6))  # Указываем размер входа (вместо placeholder)

# Применяем identity_block
A = identity_block(A_prev, f=2, filters=[2, 4, 6], stage=1, block='a')

# Создаем модель
model = tf.keras.Model(inputs=A_prev, outputs=A)

# Генерируем случайный ввод
X = np.random.randn(3, 4, 4, 6).astype(np.float32)

# Получаем выход модели
out = model(X)

# Печатаем результат
print("out = ", out[1][0])  # Измените индексы в зависимости от нужного выхода

out =  tf.Tensor(
[[0.         0.05848081 0.86156154 0.4086107  0.         0.        ]
 [0.         0.70281047 0.         0.5053233  0.         0.        ]
 [0.9939816  0.27038705 1.6661687  0.         0.         0.        ]
 [0.         0.         0.95605093 1.6211839  0.         0.        ]], shape=(4, 6), dtype=float32)


## 3 - Building our first ResNet model (50 layers)

We now have the necessary blocks to build a very deep ResNet. The following figure describes in detail the architecture of this neural network. "ID BLOCK" in the diagram stands for "Identity block," and "ID BLOCK x3" means you should stack 3 identity blocks together.

<img src="images/ResNet-50.png" style="width:850px;height:150px;">

The details of this ResNet-50 model are:
- Zero-padding pads the input with a pad of (3,3)
- Stage 1:
    - The 2D Convolution has 64 filters of shape (7,7) and uses a stride of (2,2). Its name is "conv1".
    - BatchNorm is applied to the channels axis of the input.
    - MaxPooling uses a (3,3) window and a (2,2) stride.
- Stage 2:
    - The convolutional block uses three set of filters of size [64,64,256], "f" is 3, "s" is 1 and the block is "a".
    - The 2 identity blocks use three set of filters of size [64,64,256], "f" is 3 and the blocks are "b" and "c".
- Stage 3:
    - The convolutional block uses three set of filters of size [128,128,512], "f" is 3, "s" is 2 and the block is "a".
    - The 3 identity blocks use three set of filters of size [128,128,512], "f" is 3 and the blocks are "b", "c" and "d".
- Stage 4:
    - The convolutional block uses three set of filters of size [256, 256, 1024], "f" is 3, "s" is 2 and the block is "a".
    - The 5 identity blocks use three set of filters of size [256, 256, 1024], "f" is 3 and the blocks are "b", "c", "d", "e" and "f".
- Stage 5:
    - The convolutional block uses three set of filters of size [512, 512, 2048], "f" is 3, "s" is 2 and the block is "a".
    - The 2 identity blocks use three set of filters of size [512, 512, 2048], "f" is 3 and the blocks are "b" and "c".
- The 2D Average Pooling uses a window of shape (2,2) and its name is "avg_pool".
- The flatten doesn't have any hyperparameters or name.
- The Fully Connected (Dense) layer reduces its input to the number of classes using a softmax activation. Its name should be `'fc' + str(classes)`.

So we will implement the ResNet with 50 layers described in the figure above into following architecture:<br>
    CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> CONVBLOCK -> IDBLOCK*2 -> CONVBLOCK -> IDBLOCK*3
    -> CONVBLOCK -> IDBLOCK*5 -> CONVBLOCK -> IDBLOCK*2 -> AVGPOOL -> TOPLAYER

**Arguments**:<br>
    input_shape - shape of the images of the dataset<br>
    classes - integer, number of classes<br>

**Returns**:<br>
    model - a Model() instance in Keras<br>

In [ ]:
def ResNet50(input_shape = (64, 64, 3), classes = 2):
    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)

    # Stage 1
    X = Conv2D(64, (7, 7), strides = (2, 2), name = 'conv1', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = 'bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, f = 3, filters = [64, 64, 256], stage = 2, block='a', s = 1)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')

    # Stage 3
    X = convolutional_block(X, f = 3, filters = [128, 128, 512], stage = 3, block='a', s = 2)
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='d')

    # Stage 4
    X = convolutional_block(X, f = 3, filters = [256, 256, 1024], stage = 4, block='a', s = 2)
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')

    # Stage 5
    X = convolutional_block(X, f = 3, filters = [512, 512, 2048], stage = 5, block='a', s = 2)
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')

    # AVGPOOL.
    X = AveragePooling2D((2, 2), name='avg_pool')(X)

    # output layer
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer = glorot_uniform(seed=0))(X)

    # Create model
    model = Model(inputs = X_input, outputs = X, name='ResNet50')

    return model

Run the following code to build the model's graph:

In [ ]:
model = ResNet50(input_shape = (ROWS, COLS, CHANNELS), classes = CLASSES)

Now we need to configure the learning process by compiling the model.

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
import kagglehub
import numpy as np
import seaborn as sns
import pandas as pd
import imgaug.augmenters as iaa
import imageio
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, f1_score, mean_squared_error, mean_absolute_error
from sklearn.linear_model import SGDClassifier
from PIL import Image

# Обучение

The model is now ready to be trained. Run the following cell to train your model on 100 epochs with a batch size of 64:

In [ ]:
model.fit(X_train, Y_train, epochs = 10, batch_size = 64)

Epoch 1/10
118/118 ━━━━━━━━━━━━━━━━━━━━ 922s 7s/step - accuracy: 0.6091 - loss: 1.9401
Epoch 2/10
118/118 ━━━━━━━━━━━━━━━━━━━━ 879s 7s/step - accuracy: 0.6766 - loss: 0.9902
Epoch 3/10
118/118 ━━━━━━━━━━━━━━━━━━━━ 923s 7s/step - accuracy: 0.7065 - loss: 0.8204
Epoch 4/10
118/118 ━━━━━━━━━━━━━━━━━━━━ 879s 7s/step - accuracy: 0.7197 - loss: 0.7953
Epoch 5/10
118/118 ━━━━━━━━━━━━━━━━━━━━ 925s 7s/step - accuracy: 0.6868 - loss: 1.0465
Epoch 6/10
118/118 ━━━━━━━━━━━━━━━━━━━━ 920s 7s/step - accuracy: 0.6762 - loss: 1.0245
Epoch 7/10
118/118 ━━━━━━━━━━━━━━━━━━━━ 939s 8s/step - accuracy: 0.7076 - loss: 0.8734
Epoch 8/10
118/118 ━━━━━━━━━━━━━━━━━━━━ 917s 8s/step - accuracy: 0.7141 - loss: 0.8077
Epoch 9/10
118/118 ━━━━━━━━━━━━━━━━━━━━ 902s 7s/step - accuracy: 0.7238 - loss: 0.7706
Epoch 10/10
118/118 ━━━━━━━━━━━━━━━━━━━━ 917s 7s/step - accuracy: 0.7161 - loss: 0.7896


Let's see how this model performs on the test set.

In [ ]:
preds = model.evaluate(X_test, Y_test)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

79/79 ━━━━━━━━━━━━━━━━━━━━ 32s 369ms/step - accuracy: 0.6588 - loss: 0.9233
Loss = 0.8780505657196045
Test Accuracy = 0.6761181950569153


We can also print a summary of our model by running the following code:

In [ ]:
model.summary()

Model: "ResNet50"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2             │ (None, 64, 64, 3)      │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ zero_padding2d            │ (None, 70, 70, 3)      │              0 │ input_layer_2[0][0]    │
│ (ZeroPadding2D)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1 (Conv2D)            │ (None, 32, 32, 64)     │          9,472 │ zero_padding2d[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bn_conv1                  │ (None, 32, 32, 64)     │            256 │ conv1[0][0]            │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_6 (Activation) │ (None, 32, 32, 64)     │              0 │ bn_conv1[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d             │ (None, 15, 15, 64)     │              0 │ activation_6[0][0]     │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ res2a_branch2a (Conv2D)   │ (None, 15, 15, 64)     │          4,160 │ max_pooling2d[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bn2a_branch2a             │ (None, 15, 15, 64)     │            256 │ res2a_branch2a[0][0]   │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_7 (Activation) │ (None, 15, 15, 64)     │              0 │ bn2a_branch2a[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ res2a_branch2b (Conv2D)   │ (None, 15, 15, 64)     │         36,928 │ activation_7[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bn2a_branch2b             │ (None, 15, 15, 64)     │            256 │ res2a_branch2b[0][0]   │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_8 (Activation) │ (None, 15, 15, 64)     │              0 │ bn2a_branch2b[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ res2a_branch2c (Conv2D)   │ (None, 15, 15, 256)    │         16,640 │ activation_8[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ res2a_branch1 (Conv2D)    │ (None, 15, 15, 256)    │         16,640 │ max_pooling2d[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bn2a_branch2c             │ (None, 15, 15, 256)    │          1,024 │ res2a_branch2c[0][0]   │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bn2a_branch1              │ (None, 15, 15, 256)    │          1,024 │ res2a_branch1[0][0]    │
│ (BatchNormalization) 

 Total params: 70,699,927 (269.70 MB)

 Trainable params: 23,548,935 (89.83 MB)

 Non-trainable params: 53,120 (207.50 KB)

 Optimizer params: 47,097,872 (179.66 MB)

For future use we can save our model to file:

In [ ]:
model.save('ResNet50.h5')

Finally, we can run the code below to visualize our ResNet50:

In [ ]:
plot_model(model, to_file='model.png')
SVG(model_to_dot(model).create(prog='dot', format='svg'))

NameError: name 'model_to_dot' is not defined